In [109]:
import numpy as np
import pandas as pd
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#from mpl_toolkits import mplot3d 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.svm import SVC
from scipy.sparse import csr_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from tensorflow import keras
from nltk.corpus import stopwords
from sklearn.svm import LinearSVC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Dense, Activation, MaxPool1D
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import VotingClassifier
import tensorflow as tf

from sklearn import set_config
set_config(display='diagram')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harisnaveed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [110]:
def split_labels(data, label_feature):
    """
    Split the given column of of the data, returning the full data set (without that
    feature) and the split off feature.
    """
    return data.drop(columns=label_feature), data[label_feature]

# Frame the Problem and Look at the Big Picture 

1. Define the objective in business terms. 

    The business objective is to create a machine learning model that takes a movie goer's review and determines if it is positive or negative. The client can use this to quickly determine the overall reaction to a movie showing. 

2. How will your solution be used? 

    The solution will be used by the client to quickly determine reactions to movie showings. 

3. What are the current solutions/workarounds (if any)? 

    The current solution is to manually poll viewers to determine how they felt about the movie. This is a slow process and the review can be very mixed. 

4. How should you frame this problem (supervised/unsupervised, online/offline, ...)?

    This problem is a supervised offline sentiment analysis problem. 

5. How should performance be measured? Is the performance measure aligned with the business objective? 

    Our metric for this problem will be accuracy. The most important function of this model is to accurately measure the sentiment of the review, so it is important that our model has high accuracy.

6. What would be the minimum performance needed to reach the business objective? 

    The minimum performance required is 80% accuracy. 

7. What are comparable problems? Can you reuse experience or tools? 

    A comparable problem was using by a youtuber named Micheal Reeves. He used sentiment analysis to determine if reddit posts on r/wallstreetbets were positive or negative. We can reuse the experience but not the tools he used. There are resuable tools in the textbook for this course that we will be using. 

8. Is human expertise available? 

    No human expertise is availible at this time. 

9. How would you solve the problem manually? 

    To solve this problem manually, we would interview the movie goers individually to determine their feelings about the movie. 

10.  List the assumptions you (or others) have made so far. Verify assumptions if possible. 

    - A review can either be positive or negative. There are no "meh" reviews. 

# Get the Data 

1. List the data you need and how much you need 

    We need movie reviews and a label that determines their sentiment (1 being positive, 0 being negative). We need over 1000.

2. Find and document where you can get that data 

    The data can be gathered form the open source website Kaggle.

3. Get access authorizations 

    The webiste and data is open source, so we have full access to the data.

4. Create a workspace (with enough storage space) 

    This notebook.

5. Get the data 

    The data was downloaded from kaggle and inserted into this repository.
    Link: https://www.kaggle.com/datasets/yasserh/imdb-movie-ratings-sentiment-analysis?resource=download

6. Convert the data to a format you can easily manipulate (without changing the data itself) 

    Made into a pandas dataframe.

7. Ensure sensitive information is deleted or protected (e.g. anonymized) 

    Not applicable to this situation.

8. Check the size and type of data (time series, geographical, ...) 

    - text: object: 40000 entries
    - label: int: 400000 entries
    
9. Sample a test set, put it aside, and never look at it (no data snooping!) 

In [111]:
data = pd.read_csv("movie.csv")

In [112]:
data

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [113]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [114]:
train_set, test_set = train_test_split(data, stratify=data['label'], test_size=0.2, random_state=420)

# Explore the Data 

1. Copy the data for exploration, downsampling to a manageable size if necessary. 


2. Study each attribute and its characteristics: Name; Type (categorical, numerical, bounded, text, structured, ...); % of missing values; Noisiness and type of noise (stochastic, outliers, rounding errors, ...); Usefulness for the task; Type of distribution (Gaussian, uniform logarithmic, ...) 

    - text is a object datatype with no missing data.
    - label is an interger value of either 1 or a 0. 1 is positive, 0 is negative. There is no missing data. In the data set, there is almost an equal number of positive and negative labels.


3. For supervised learning tasks, identify the target attribute(s) 

    The target attribute is the label feature. It is the sentiment of the text.

4. Visualize the data 


5. Study the correlations between attributes 


6. Study how you would solve the problem manually 


7. Identify the promising transformations you may want to apply 


8. Identify extra data that would be useful (go back to “Get the Data”) 


9. Document what you have learned 
    - With statification, there is almost an equal number of text with positive/negative sentiment.

In [115]:
train_copy = train_set.copy()

In [116]:
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32000 entries, 33632 to 13973
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    32000 non-null  object
 1   label   32000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 750.0+ KB


In [117]:
train_copy['label'].value_counts()

0    16015
1    15985
Name: label, dtype: int64

# Bag of Words

In [118]:
all_stop_words = set(stopwords.words('english'))
all_stop_words.remove('not')
ps = PorterStemmer()


In [119]:
def BagOfWords(train_copy):
    corpus = []
    for text in train_copy['text']:
        review = text.lower()
        review = re.sub("<.+>|[^a-zA-Z]|z.+z", " ", review)
        review = review.split()
        review = [word for word in review if word not in all_stop_words]
        review = " ".join(ps.stem(word) for word in review)
        corpus.append(review)
    return corpus
    


In [120]:
corpus = BagOfWords(train_copy)
corpus

['best bit film alan pull knicker ran cut throat ra watch girl find pull knicker',
 'rain shine outsid enter movi hous make happi not come right light go settl bar ice cream move pictur begin flicker screen feel content dark back begin time sit around campfir look modern version flicker flame time per second share joy discov unknown turn twist scenario rest clan spectat not happi not write comment long live romant comedi',
 'memor line short live show view episod line introduc fratern intramur flag footbal team start use line break huddl offens instead readi break quarter back said footbal rest squad respond bet fun way break huddl oppon scratch head watch show use line ad uniqu element season never forget best season time colleg year small way due fun use line show pretti much stinker live memori pi kappa phi intramur flag footbal squad west virginia tech',
 'dawn realli surpris saw far less gruesom horror adventur expect instead director jeff lieberman conjur wonder evoc disquiet atm

In [121]:
pop_words = {}
for line in corpus:
    words = line.split()
    for word in words:
        if pop_words.get(word, -1) == -1:
            pop_words[word] = 1
        else:
            pop_words[word] += 1
pop_words

{'best': 5031,
 'bit': 2282,
 'film': 37370,
 'alan': 173,
 'pull': 634,
 'knicker': 11,
 'ran': 167,
 'cut': 903,
 'throat': 94,
 'ra': 39,
 'watch': 12654,
 'girl': 2657,
 'find': 3785,
 'rain': 175,
 'shine': 263,
 'outsid': 359,
 'enter': 274,
 'movi': 44850,
 'hous': 1473,
 'make': 10684,
 'happi': 776,
 'not': 22354,
 'come': 4567,
 'right': 2402,
 'light': 1066,
 'go': 6652,
 'settl': 90,
 'bar': 283,
 'ice': 222,
 'cream': 71,
 'move': 1674,
 'pictur': 1420,
 'begin': 1847,
 'flicker': 20,
 'screen': 1966,
 'feel': 3626,
 'content': 267,
 'dark': 1177,
 'back': 3463,
 'time': 12122,
 'sit': 944,
 'around': 2262,
 'campfir': 17,
 'look': 7014,
 'modern': 634,
 'version': 1717,
 'flame': 68,
 'per': 104,
 'second': 1482,
 'share': 345,
 'joy': 245,
 'discov': 545,
 'unknown': 204,
 'turn': 2490,
 'twist': 881,
 'scenario': 172,
 'rest': 1204,
 'clan': 28,
 'spectat': 42,
 'write': 1566,
 'comment': 1399,
 'long': 2551,
 'live': 3037,
 'romant': 643,
 'comedi': 3030,
 'memor': 482

In [122]:
keys_greater_than = []
for key in pop_words.keys():
    if pop_words.get(key) >= 1000:
        keys_greater_than.append([pop_words.get(key), key])
    
for i in sorted(keys_greater_than, reverse=True):
    print(i)

[44850, 'movi']
[37370, 'film']
[22354, 'not']
[20905, 'one']
[16688, 'like']
[12654, 'watch']
[12122, 'time']
[12073, 'good']
[11434, 'see']
[10684, 'make']
[9644, 'get']
[9190, 'would']
[9016, 'stori']
[9012, 'even']
[8786, 'realli']
[8601, 'charact']
[7881, 'well']
[7648, 'great']
[7480, 'show']
[7369, 'bad']
[7200, 'love']
[7014, 'look']
[6998, 'much']
[6715, 'peopl']
[6658, 'think']
[6652, 'go']
[6641, 'end']
[6472, 'first']
[6417, 'act']
[6309, 'scene']
[6259, 'made']
[5969, 'way']
[5775, 'seen']
[5691, 'thing']
[5691, 'could']
[5684, 'also']
[5603, 'say']
[5562, 'play']
[5316, 'know']
[5140, 'ever']
[5091, 'year']
[5031, 'best']
[4975, 'plot']
[4965, 'actor']
[4951, 'want']
[4938, 'mani']
[4680, 'better']
[4671, 'work']
[4604, 'seem']
[4588, 'never']
[4567, 'come']
[4507, 'life']
[4493, 'littl']
[4448, 'two']
[4341, 'tri']
[4326, 'take']
[4312, 'give']
[4163, 'still']
[3798, 'enjoy']
[3785, 'find']
[3765, 'man']
[3742, 'lot']
[3732, 'someth']
[3684, 'director']
[3626, 'feel']
[3

# TOP WORDS:
    [44906, 'movi']
    [37438, 'film']
    [22288, 'not']
    [20872, 'one']
    [16604, 'like']
    [12653, 'watch']
    [12103, 'time']
    [12071, 'good']
    [11416, 'see']
    [10646, 'make']
    [9604, 'get']
    [9149, 'would']
    [9014, 'stori']
    [8986, 'even']
    [8766, 'realli']
    [8552, 'charact']
    [7854, 'well']
    [7624, 'great']
    [7469, 'show']
    [7382, 'bad']
    [7205, 'love']
    [7001, 'look']
    [6975, 'much']
    [6714, 'peopl']
    [6649, 'go']

In [123]:
CountVec = CountVectorizer()
x = CountVec.fit_transform(corpus).toarray()

In [124]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [125]:
occurances = CountVec.vocabulary_
occurances

{'best': 3534,
 'bit': 3822,
 'film': 13355,
 'alan': 745,
 'pull': 30417,
 'knicker': 20837,
 'ran': 30943,
 'cut': 8768,
 'throat': 38375,
 'ra': 30739,
 'watch': 41804,
 'girl': 15213,
 'find': 13397,
 'rain': 30852,
 'shine': 34361,
 'outsid': 27730,
 'enter': 12020,
 'movi': 25550,
 'hous': 17868,
 'make': 23157,
 'happi': 16568,
 'not': 26823,
 'come': 7469,
 'right': 32037,
 'light': 22053,
 'go': 15376,
 'settl': 34005,
 'bar': 2790,
 'ice': 18216,
 'cream': 8366,
 'move': 25545,
 'pictur': 29053,
 'begin': 3247,
 'flicker': 13640,
 'screen': 33570,
 'feel': 13110,
 'content': 7855,
 'dark': 9035,
 'back': 2508,
 'time': 38519,
 'sit': 34846,
 'around': 1886,
 'campfir': 5562,
 'look': 22455,
 'modern': 25083,
 'version': 41142,
 'flame': 13555,
 'per': 28668,
 'second': 33705,
 'share': 34180,
 'joy': 19972,
 'discov': 10198,
 'unknown': 40319,
 'turn': 39511,
 'twist': 39601,
 'scenario': 33296,
 'rest': 31770,
 'clan': 6988,
 'spectat': 35815,
 'write': 42783,
 'comment': 75

In [126]:
keys_greater_than = []
for key in occurances.keys():
    if occurances.get(key) >= 10000:
        keys_greater_than.append((occurances.get(key), key))


In [127]:
for i in sorted(keys_greater_than):
    print(i)

(10000, 'digicorp')
(10001, 'digimon')
(10002, 'digisoft')
(10003, 'digit')
(10004, 'digitech')
(10005, 'digitis')
(10006, 'digiulio')
(10007, 'digiview')
(10008, 'dignifi')
(10009, 'digniti')
(10010, 'digress')
(10011, 'digust')
(10012, 'dike')
(10013, 'dikker')
(10014, 'diklaja')
(10015, 'dil')
(10016, 'dilapid')
(10017, 'dilbert')
(10018, 'dildo')
(10019, 'dilemma')
(10020, 'dilettant')
(10021, 'dilettantish')
(10022, 'dilig')
(10023, 'dilip')
(10024, 'dillan')
(10025, 'dillemma')
(10026, 'diller')
(10027, 'dilli')
(10028, 'dilling')
(10029, 'dillman')
(10030, 'dillon')
(10031, 'dillus')
(10032, 'dillut')
(10033, 'dilophosauru')
(10034, 'dilut')
(10035, 'dim')
(10036, 'dima')
(10037, 'dimaggio')
(10038, 'dime')
(10039, 'dimeco')
(10040, 'dimens')
(10041, 'dimension')
(10042, 'dimensionless')
(10043, 'dimestor')
(10044, 'diminish')
(10045, 'diminut')
(10046, 'dimitri')
(10047, 'dimitrio')
(10048, 'dimitru')
(10049, 'dimli')
(10050, 'dimmer')
(10051, 'dimmest')
(10052, 'dimpl')
(10053

In [128]:
#plt.plot(keys_greater_than, values)
print(keys_greater_than[0:10])


[(13355, 'film'), (30417, 'pull'), (20837, 'knicker'), (30943, 'ran'), (38375, 'throat'), (30739, 'ra'), (41804, 'watch'), (15213, 'girl'), (13397, 'find'), (30852, 'rain')]


In [129]:
print(x.shape)
print(type(x))

(32000, 43381)
<class 'numpy.ndarray'>


# Prepare the Data 
1. Data cleaning: Fix/remove outliers (optional); Fill in missing values (with 0, mean, 
median...) or drop rows/columns 
2. Feature selection (optional): Drop attributes that provide no useful information 
for the task 
3. Feature engineering, where appropriate: Discretize continuous features; Decompose features (categorical, date/time, ...), 
Add promising transformations of features (log(𝑥𝑥), √𝑥𝑥, 𝑥𝑥2, ...); Aggregate features into promising new features 
4. Feature scaling: standardize or normalize features 

In [130]:
X, y = split_labels(train_set, 'label',)

In [131]:
# Creating a class that lowercase the sentences, removing <br> cases, and removing instances like zzzzzz
all_stop_words = stopwords.words('english')
all_stop_words.remove('not')
ps = PorterStemmer()

def text_editer(text):
    review = text.lower()
    review = re.sub("<.+>|[^a-zA-Z]|z.+z", " ", review)
    review = review.split()
    review = [word for word in review if word not in set(all_stop_words)]
    review = " ".join(ps.stem(word) for word in review)
    return review

class EditingText(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        super().__init__()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        print(type(X))
        #X = pd.DataFrame(X)
        data = np.array([text_editer(text) for text in X['text']]).flatten()
        #data = data.to_numpy().reshape(-1,1)
        return data

In [132]:
preprocessor = Pipeline(steps=[
    ('EditingText', EditingText()),
    ('TfidfVectorizer', TfidfVectorizer()),
    ('TfidfVectorizerTransformer', TfidfTransformer()),
    ('StandardScaler', StandardScaler(with_mean=False)),
])

In [133]:
preprocessor.fit(X,y)

<class 'pandas.core.frame.DataFrame'>


Pipeline(steps=[('EditingText', EditingText()),
                ('TfidfVectorizer', TfidfVectorizer()),
                ('TfidfVectorizerTransformer', TfidfTransformer()),
                ('StandardScaler', StandardScaler(with_mean=False))])

In [134]:
transformed = preprocessor.transform(X)

<class 'pandas.core.frame.DataFrame'>


In [135]:
transformed

<32000x43381 sparse matrix of type '<class 'numpy.float64'>'
	with 1749125 stored elements in Compressed Sparse Row format>

# Short-List Promising Models 

In [136]:
X, y = split_labels(train_set, 'label')

In [137]:
y

33632    1
11432    1
16188    0
31733    1
29102    0
        ..
37424    1
24370    0
22580    0
29688    0
13973    0
Name: label, Length: 32000, dtype: int64

In [138]:
MultiNB = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('MultiNB', MultinomialNB())
])
MultiNB.fit(X,y)

<class 'pandas.core.frame.DataFrame'>


Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('EditingText', EditingText()),
                                 ('TfidfVectorizer', TfidfVectorizer()),
                                 ('TfidfVectorizerTransformer',
                                  TfidfTransformer()),
                                 ('StandardScaler',
                                  StandardScaler(with_mean=False))])),
                ('MultiNB', MultinomialNB())])

In [139]:
y_pred = cross_val_predict(MultiNB, X, y)
accuracy_score(y,y_pred)

<class 'pandas.core.frame.DataFrame'>


KeyboardInterrupt: 

In [ ]:
BernNB = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('BernouliNB', BernoulliNB())
])
BernNB.fit(X,y)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('EditingText', EditingText()),
                                 ('TfidfVectorizer', TfidfVectorizer()),
                                 ('TfidfVectorizerTransformer',
                                  TfidfTransformer()),
                                 ('StandardScaler',
                                  StandardScaler(with_mean=False))])),
                ('BernouliNB', BernoulliNB())])

In [ ]:
y_pred = cross_val_predict(BernNB, X, y)
accuracy_score(y,y_pred)    

0.83728125

In [ ]:
scores = cross_val_score(BernNB, X, y, scoring='accuracy')
scores

array([0.84      , 0.8371875 , 0.83546875, 0.8371875 , 0.8365625 ])

In [ ]:
SVC = Pipeline([
    ('preprocessor', preprocessor),
    ('SVM', LinearSVC(loss="hinge", C=1.0)),
])
SVC.fit(X, y)
y_pred = SVC.predict(X)
accuracy_score(y, y_pred)

#Severe Overfitting

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.99946875

In [ ]:
csr_matrix.toarray(transformed)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
transformed_tensor = tf.convert_to_tensor(csr_matrix.toarray(transformed))
transformed_tensor

<tf.Tensor: shape=(32000, 43381), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])>

In [ ]:
X, y = split_labels(train_set, 'label')
X = tf.convert_to_tensor(X)
y = tf.convert_to_tensor(y)
X_test, X_train = X[:5000], X[5000:]
y_test, y_train = y[:5000], y[5000:]
train_set_np = X_train
print(X_test.shape, X_train.shape)
print(y_test.shape, y_train.shape)
print(type(X_test))

(5000, 1) (27000, 1)
(5000,) (27000,)
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [ ]:
VOCAB_SIZE = 1000
encoder = keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(transformed_tensor.map(lambda text, label: text))

model = keras.Sequential()
model.add(encoder)
model.add(keras.layers.Flatten(input_shape=X_train[0].shape, name="input"))
model.add(keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(64,  return_sequences=True)))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(2, activation=keras.activations.softmax, name="output"))

model.build(X_test.shape)
model.summary()



In [ ]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Fine-Tune the System 
1. Fine-tune the hyperparameters using cross-validation. Treat your data 
transformation choices as hyperparameters, especially when you are not sure 
about them. Unless there are very few hyperparameter values to explore, prefer 
random search over grid search. If training is very long, you may prefer a Bayesian 
optimization approach. 
2. Try Ensemble methods. Combining your best models will often perform better 
than running them individually. 
3. Once you are confident about your final model, measure its performance on the 
test set to estimate the generalization error. 

In [ ]:
final_pipeline = VotingClassifier(
    estimators=[('MultiNB', MultiNB), ('BernNB', BernNB)],
    voting='hard')
final_pipeline.fit(X, y)

scores = cross_val_score(final_pipeline, X, y, scoring='accuracy', cv=5)
scores

In [ ]:
final_pipeline.get_params().keys() # Seeing the param keys of the pipeline.